In [1]:
import torch
from pathlib import Path
import numpy as np
import scipy

In [2]:
root_dir = Path.cwd().parent
data_dir = root_dir / 'data'
model_dir = root_dir / 'models'

root_dir
data_dir

WindowsPath('c:/Users/jakab/Documents/University/DTU/2023_spring/Perception_for_AS/Final_project/PFAS---final-project/data')

In [3]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_dir / 'best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\jakab/.cache\torch\hub\master.zip
YOLOv5  2023-4-30 Python-3.11.2 torch-2.0.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients


requirements: C:\Users\jakab\.cache\torch\hub\requirements.txt not found, check failed.


Adding AutoShape... 


In [4]:
model.eval();

In [5]:

# Images
img1_path = data_dir / "raw/final_project_2023_rect/seq_03/image_02/data/0000000000.png"  # or file, Path, PIL, OpenCV, numpy, list
img2_path = data_dir / "raw/final_project_2023_rect/seq_03/image_03/data/0000000000.png"

# Inference
results = model([img1_path, img2_path])

# Results
r = results.pandas().xyxy # or .show(), .save(), .crop(), .pandas(), etc.

In [6]:
r[0].head()

,xmin,ymin,xmax,ymax,confidence,class,name
0,0.000000,198.781540,302.233887,366.846100,0.982934,0,Car
1,446.705200,168.243835,481.563446,192.657181,0.948782,0,Car
2,991.425781,143.852783,1028.824341,215.211456,0.941393,2,Cyclist
3,1073.351807,147.619415,1195.596802,186.312271,0.941104,0,Car
4,553.656372,152.106049,618.121643,314.763367,0.922514,1,Pedestrian


In [7]:
r[1].head()

,xmin,ymin,xmax,ymax,confidence,class,name
0,0.000000,197.421494,234.741684,367.471191,0.982592,0,Car
1,970.142761,148.914749,1007.361328,217.774231,0.945694,2,Cyclist
2,438.436859,168.651886,473.241882,192.590729,0.944815,0,Car
3,1061.583130,147.692871,1190.861816,186.746582,0.920443,0,Car
4,882.492981,141.664017,945.607788,292.411469,0.878738,1,Pedestrian


In [8]:
import cv2
frame = cv2.imread(str(img1_path))
for _, box in r[0].iterrows():
    if box["class"]==0:
        xB = int(box["xmin"])
        yB = int(box["ymin"])
        xA = int(box["xmax"])
        yA = int(box["ymax"])
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
cv2.imshow('im', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
r1 = np.asarray(r[0])
r2 = np.asarray(r[1])
img1 = cv2.imread(str(img1_path))
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread(str(img2_path))
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
nb_matches = 20
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()
im_height = img1.shape[0]
match_matrix = np.zeros((r1.shape[0], r2.shape[0]))
for i1, bbox1 in enumerate(r1):
    for i2, bbox2 in enumerate(r2):
        cy1 = bbox1[1]+bbox1[3]/2
        cy2 = bbox2[1]+bbox2[3]/2
        if abs(cy1-cy2)/im_height <= 0.1 and bbox1[5] == bbox2[5]:
            bbox1_im = img1_gray[int(bbox1[1]):int(bbox1[3]),int(bbox1[0]):int(bbox1[2])]
            bbox2_im = img2_gray[int(bbox2[1]):int(bbox2[3]),int(bbox2[0]):int(bbox2[2])]
            kp1, des1 = sift.detectAndCompute(bbox1_im, None)
            kp2, des2 = sift.detectAndCompute(bbox2_im, None)
            matches = bf.match(des1, des2)
            matches = sorted(matches, key = lambda x:x.distance)[:nb_matches]
            for m in matches:
                match_matrix[i1,i2] += m.distance
        else:
            match_matrix[i1,i2] = np.inf
row_ind, col_ind = scipy.optimize.linear_sum_assignment(match_matrix)

In [11]:
for i1,i2 in zip(row_ind, col_ind):
    random_color = tuple(int(color) for color in np.random.choice(range(255),size=3))
    box1 = r1[i1]
    x_min = int(box1[0])
    y_min = int(box1[1])
    x_max = int(box1[2])
    y_max = int(box1[3])
    cv2.rectangle(img1, (x_min, y_min), (x_max, y_max), random_color, 2)
    box2 = r2[i2]
    x_min = int(box2[0])
    y_min = int(box2[1])
    x_max = int(box2[2])
    y_max = int(box2[3])
    cv2.rectangle(img2, (x_min, y_min), (x_max, y_max), random_color, 2)
merged = np.concatenate((img1, img2), axis=0)
cv2.imshow('im', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()